# Config generator for Home Assistant

In [10]:
import yaml

## Stale automations

Detect stale sensor readings and shut down associated thermostats

Example:

```yaml
- id: '1604254268934'
  alias: 'Stale sensor: Pip''s Room'
  description: ''
  trigger:
    - platform: time_pattern
      minutes: 7
  condition:
  - condition: and
    conditions:
    - condition: template
      value_template: '{{ (as_timestamp(now()) - as_timestamp(states.sensor.pip_temperature.last_changed))/(60*60) > 12 }}'
    - condition: state
      entity_id: automation.turn_pip_s_heating_on
      state: 'on'
  action:
  - data: {}
    entity_id: automation.turn_pip_s_heating_on
    service: automation.turn_off
  - data:
      message: Disabling Pip Heating
      title: 'Stale sensor: pip'
    service: notify.mobile_app_pixel_2
  mode: single
```

Or, using more recently created ["last changed" value](real_last_change.ipynb):

```yaml
  - condition: numeric_state
    entity_id: sensor.rlc_attic_office_temperature
    below: '2'
```

### Simple sensors 1:1 control automation

These have a single sensor and a single automation that turns on the heating.

- When stale, turn off the automation.
- When fresh, turn on the automation.

In [23]:
# escape any ' with ''
settings = [
    {"id": "pip_room_stale", 
     "alias": "Pip''s room", 
     "sensor": "pip_temperature", 
     "rlc_sensor": "rlc_pip_temperature",
     "timeout": 4, 
     "automation": "turn_pip_s_heating_on"},
    {"id": "kitchen_stale",
     "alias": "Kitchen", 
     "sensor": "kitchen_temperature", 
     "rlc_sensor": "rlc_kitchen_temperature",
     "timeout": 4, 
     "automation": "kitchen_heating_on"}
]

In [26]:
config=""
for item in settings:
    config += f"""
- id: '{item["id"]}'
  alias: 'Stale sensor: {item["alias"]}'
  description: ''
  trigger:
    - platform: time_pattern
      minutes: 7
  condition:
  - condition: and
    conditions:
    - condition: numeric_state
      entity_id: sensor.{item["rlc_sensor"]}
      above: '{item["timeout"]}'
    - condition: state
      entity_id: automation.{item["automation"]}
      state: 'on'
  action:
  - data: {{}}
    entity_id: automation.{item["automation"]}
    service: automation.turn_off
  - data:
      message: Disabling {item["alias"]}
      title: 'Stale sensor: {item["sensor"]}'
    service: notify.mobile_app_pixel_2
  mode: single
"""
# - id: '{item["id"]}_enable'
#   alias: 'Fresh sensor: {item["alias"]}'
#   description: ''
#   trigger:
#     - platform: time_pattern
#       minutes: 8
#   condition:
#   - condition: and
#     conditions:
#     - condition: numeric_state
#       entity_id: sensor.{item["rlc_sensor"]}
#       below: '{item["timeout"]}'
#     - condition: state
#       entity_id: automation.{item["automation"]}
#       state: 'off'
#   action:
#   - data: {{}}
#     entity_id: automation.{item["automation"]}
#     service: automation.turn_on
#   - data:
#       message: Enabling {item["alias"]}
#       title: 'Fresh sensor: {item["sensor"]}'
#     service: notify.mobile_app_pixel_2
#   mode: single

try:
    yaml.safe_load(config)
    print(config)
except:
    print("YAML failed to parse")


- id: 'pip_room_stale'
  alias: 'Stale sensor: Pip''s room'
  description: ''
  trigger:
    - platform: time_pattern
      minutes: 7
  condition:
  - condition: and
    conditions:
    - condition: numeric_state
      entity_id: sensor.rlc_pip_temperature
      above: '4'
    - condition: state
      entity_id: automation.turn_pip_s_heating_on
      state: 'on'
  action:
  - data: {}
    entity_id: automation.turn_pip_s_heating_on
    service: automation.turn_off
  - data:
      message: Disabling Pip''s room
      title: 'Stale sensor: pip_temperature'
    service: notify.mobile_app_pixel_2
  mode: single

- id: 'kitchen_stale'
  alias: 'Stale sensor: Kitchen'
  description: ''
  trigger:
    - platform: time_pattern
      minutes: 7
  condition:
  - condition: and
    conditions:
    - condition: numeric_state
      entity_id: sensor.rlc_kitchen_temperature
      above: '4'
    - condition: state
      entity_id: automation.kitchen_heating_on
      state: 'on'
  action:
  - data: 

### Simple sensors 1:1 control thermostat

These have a single sensor and a single thermostat that is normally on all the time.

- When stale, turn off the thermostat.
- When fresh, turn on the automation.

```yaml

    - condition: state
      entity_id: climate.tack_room
      state: heat
      
  - data: {}
    entity_id: climate.tack_room
    service: climate.turn_off
```

In [32]:
# escape any ' with ''
settings = [
    {"id": "tackroom_stale", 
     "alias": "Tackroom", 
     "sensor": "tackroom_temperature",
     "rlc_sensor": "rlc_tackroom_temperature",
     "timeout": 4, 
     "climate": "tack_room"},
    {"id": "greenhouse_stale",
     "alias": "Greenhouse", 
     "sensor": "greenhouse_temperature",
     "rlc_sensor": "rlc_greenhouse_temperature",
     "timeout": 4, 
     "climate": "greenhouse"}
]

In [33]:
config=""
for item in settings:
    config += f"""
- id: '{item["id"]}'
  alias: 'Stale sensor: {item["alias"]}'
  description: ''
  trigger:
    - platform: time_pattern
      minutes: 7
  condition:
  - condition: and
    conditions:
    - condition: numeric_state
      entity_id: sensor.{item["rlc_sensor"]}
      above: '{item["timeout"]}'
    - condition: state
      entity_id: climate.{item["climate"]}
      state: heat
  action:
  - data: {{}}
    entity_id: climate.{item["climate"]}
    service: climate.turn_off
  - data:
      message: Disabling {item["alias"]}
      title: 'Stale sensor: {item["sensor"]}'
    service: notify.mobile_app_pixel_2
  mode: single
"""
# - id: '{item["id"]}_enable'
#   alias: 'Fresh sensor: {item["alias"]}'
#   description: ''
#   trigger:
#     - platform: time_pattern
#       minutes: 8
#   condition:
#   - condition: and
#     conditions:
#     - condition: numeric_state
#       entity_id: sensor.{item["rlc_sensor"]}
#       below: '{item["timeout"]}'
#     - condition: state
#       entity_id: automation.{item["automation"]}
#       state: 'off'
#   action:
#   - data: {{}}
#     entity_id: automation.{item["automation"]}
#     service: automation.turn_on
#   - data:
#       message: Enabling {item["alias"]}
#       title: 'Fresh sensor: {item["sensor"]}'
#     service: notify.mobile_app_pixel_2
#   mode: single

try:
    yaml.safe_load(config)
    print(config)
except:
    print("YAML failed to parse")


- id: 'tackroom_stale'
  alias: 'Stale sensor: Tackroom'
  description: ''
  trigger:
    - platform: time_pattern
      minutes: 7
  condition:
  - condition: and
    conditions:
    - condition: numeric_state
      entity_id: sensor.rlc_tackroom_temperature
      above: '4'
    - condition: state
      entity_id: climate.tack_room
      state: heat
  action:
  - data: {}
    entity_id: climate.tack_room
    service: climate.turn_off
  - data:
      message: Disabling Tackroom
      title: 'Stale sensor: tackroom_temperature'
    service: notify.mobile_app_pixel_2
  mode: single

- id: 'greenhouse_stale'
  alias: 'Stale sensor: Greenhouse'
  description: ''
  trigger:
    - platform: time_pattern
      minutes: 7
  condition:
  - condition: and
    conditions:
    - condition: numeric_state
      entity_id: sensor.rlc_greenhouse_temperature
      above: '4'
    - condition: state
      entity_id: climate.greenhouse
      state: heat
  action:
  - data: {}
    entity_id: climate.green

### Simple sensors 1:many control automation

These have a single sensor and two or more automations that turns on the heating.

- When stale, turn off the automations.
- When fresh, turn on the automations.

In [28]:
# escape any ' with ''
settings = [
    {"id": "attic_office_stale", 
     "alias": "Attic Office", 
     "sensor": "attic_office_temperature",
     "rlc_sensor": "rlc_attic_office_temperature",
     "timeout": 4, 
     "automations": ["attic_heating_on","attic_heating_on_f"]}  
]

In [29]:
config=""
for item in settings:
    config += f"""
- id: '{item["id"]}'
  alias: 'Stale sensor: {item["alias"]}'
  description: ''
  trigger:
    - platform: time_pattern
      minutes: 7
  condition:
  - condition: and
    conditions:
    - condition: numeric_state
      entity_id: sensor.{item["rlc_sensor"]}
      above: '{item["timeout"]}'
    - condition: or
      conditions:
"""
    for automation in item["automations"]:
        config += f"""
      - condition: state
        entity_id: automation.{automation}
        state: 'on'
"""
    config += f"""
  action:
"""
    for automation in item["automations"]:
        config += f"""
  - data: {{}}
    entity_id: automation.{automation}
    service: automation.turn_off
"""
    config += f"""
  - data:
      message: Disabling {item["alias"]}
      title: 'Stale sensor: {item["sensor"]}'
    service: notify.mobile_app_pixel_2
  mode: single
"""

# - id: '{item["id"]}_enable'
#   alias: 'Fresh sensor: {item["alias"]}'
#   description: ''
#   trigger:
#     - platform: time_pattern
#       minutes: 8
#   condition:
#   - condition: and
#     conditions:
#     - condition: template
#       value_template: '{{{{ (as_timestamp(now()) - as_timestamp(states.sensor.{item["sensor"]}.last_changed))/(60*60) < {item["timeout"]} }}}}'
#     - condition: or
#     conditions:
# """
#     for automation in item["automations"]:
#         config += f"""
#     - condition: state
#       entity_id: automation.{automation}
#       state: 'off'
# """
#     config += f"""
#   action:
# """
#     for automation in item["automations"]:
#         config += f"""
#   - data: {{}}
#     entity_id: automation.{automation}
#     service: automation.turn_on
# """
#     config += f"""  - data:
#       message: Enabling {item["alias"]}
#       title: 'Fresh sensor: {item["sensor"]}'
#     service: notify.mobile_app_pixel_2
#   mode: single

    
try:
    yaml.safe_load(config)
    print(config)
except:
    print("YAML failed to parse")


- id: 'attic_office_stale'
  alias: 'Stale sensor: Attic Office'
  description: ''
  trigger:
    - platform: time_pattern
      minutes: 7
  condition:
  - condition: and
    conditions:
    - condition: numeric_state
      entity_id: sensor.rlc_attic_office_temperature
      above: '4'
    - condition: or
      conditions:

      - condition: state
        entity_id: automation.attic_heating_on
        state: 'on'

      - condition: state
        entity_id: automation.attic_heating_on_f
        state: 'on'

  action:

  - data: {}
    entity_id: automation.attic_heating_on
    service: automation.turn_off

  - data: {}
    entity_id: automation.attic_heating_on_f
    service: automation.turn_off

  - data:
      message: Disabling Attic Office
      title: 'Stale sensor: attic_office_temperature'
    service: notify.mobile_app_pixel_2
  mode: single



## One Ring to Rule Them All


In [65]:
# escape any ' with ''
settings = [
    {"id": "attic_office_stale", 
     "alias": "Attic Office", 
     "sensor": "attic_office_temperature",
     "rlc_sensor": "rlc_attic_office_temperature",
     "timeout": 4, 
     "automations": ["attic_heating_on","attic_heating_on_f"]},
    {"id": "pip_room_stale", 
     "alias": "Pip''s room", 
     "sensor": "pip_temperature", 
     "rlc_sensor": "rlc_pip_temperature",
     "timeout": 4, 
     "automations": ["turn_pip_s_heating_on"]},
    {"id": "kitchen_stale",
     "alias": "Kitchen", 
     "sensor": "kitchen_temperature", 
     "rlc_sensor": "rlc_kitchen_temperature",
     "timeout": 4, 
     "automations": ["kitchen_heating_on"]},
    {"id": "tackroom_stale", 
     "alias": "Tackroom", 
     "sensor": "tackroom_temperature",
     "rlc_sensor": "rlc_tackroom_temperature",
     "timeout": 4, 
     "climates": ["tack_room"]},
    {"id": "greenhouse_stale",
     "alias": "Greenhouse", 
     "sensor": "greenhouse_temperature",
     "rlc_sensor": "rlc_greenhouse_temperature",
     "timeout": 4, 
     "climates": ["greenhouse"]},
    {"id": "hot_water_stale", 
     "alias": "Hot Water", 
     "sensor": "hot_water",
     "rlc_sensor": "rlc_hot_water",
     "timeout": 4, 
     "automations": ["hot_water_bath","hot_water"]}
]

config=""
for item in settings:
    if "automations" in item:
        is_automation = True
        is_climate = False
    elif "climates" in item:
        is_automation = False
        is_climate = True        
    else:
        raise Exception(f"No automations or climates defined for {item['id']}")
        
    config += f"""
- id: '{item["id"]}'
  alias: 'Stale sensor: {item["alias"]}'
  description: ''
  trigger:
    - platform: time_pattern
      minutes: 7
  condition:
  - condition: and
    conditions:
    - condition: numeric_state
      entity_id: sensor.{item["rlc_sensor"]}
      above: '{item["timeout"]}'
"""

    if (is_automation and len(item["automations"]) > 1) or is_climate and len(item["climates"]) >1:
        config += f"""
    - condition: or
      conditions:
"""
        if is_automation:
            for automation in item["automations"]:
                config += f"""
      - condition: state
        entity_id: automation.{automation}
        state: 'on'
"""
        elif is_climate:
            for climate in item["climates"]:
                config+= f"""
      - condition: state
        entity_id: climate.{item["climate"]}
        state: heat
"""
    elif is_automation:
        config+=f"""
    - condition: state
      entity_id: automation.{item["automations"][0]}
      state: 'on'
"""
    elif is_climate:
        config+=f"""
    - condition: state
      entity_id: climate.{item["climates"][0]}
      state: heat        
"""

    config += f"""
  action:
"""
    if is_automation:
        for automation in item["automations"]:
            config += f"""
  - data: {{}}
    entity_id: automation.{automation}
    service: automation.turn_off
"""
    elif is_climate:
        for climate in item["climates"]:
            config += f"""
  - data: {{}}
    entity_id: climate.{automation}
    service: climate.turn_off
"""
            
    config += f"""
  - data:
      message: Disabling {item["alias"]}
      title: 'Stale sensor: {item["sensor"]}'
    service: notify.mobile_app_pixel_2
  mode: single
"""

try:
    yaml.safe_load(config)
    print(config)
except:
    print("YAML failed to parse")


- id: 'attic_office_stale'
  alias: 'Stale sensor: Attic Office'
  description: ''
  trigger:
    - platform: time_pattern
      minutes: 7
  condition:
  - condition: and
    conditions:
    - condition: numeric_state
      entity_id: sensor.rlc_attic_office_temperature
      above: '4'

    - condition: or
      conditions:

      - condition: state
        entity_id: automation.attic_heating_on
        state: 'on'

      - condition: state
        entity_id: automation.attic_heating_on_f
        state: 'on'

  action:

  - data: {}
    entity_id: automation.attic_heating_on
    service: automation.turn_off

  - data: {}
    entity_id: automation.attic_heating_on_f
    service: automation.turn_off

  - data:
      message: Disabling Attic Office
      title: 'Stale sensor: attic_office_temperature'
    service: notify.mobile_app_pixel_2
  mode: single

- id: 'pip_room_stale'
  alias: 'Stale sensor: Pip''s room'
  description: ''
  trigger:
    - platform: time_pattern
      minutes: